<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/algorithm_comparisons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install surprise

In [0]:
%tensorflow_version 2.x
import numpy as np
import pandas as pd
import random

from surprise import accuracy, AlgoBase, BaselineOnly, Dataset, NormalPredictor, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, train_test_split

my_seed = 8182868
random.seed(my_seed)
np.random.seed(my_seed)

In [0]:
train = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/sagemaker/train_sagemaker_one_percent.csv")[["userID", "itemID", "my_score"]]

In [0]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(train, reader)
trainset, testset = train_test_split(data)

In [7]:
%%time
algo = NMF()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

RMSE: 2.4673
MAE:  2.0805
CPU times: user 13 s, sys: 20.3 ms, total: 13 s
Wall time: 13.1 s


In [22]:
%%time
algo = NormalPredictor()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

RMSE: 2.3440
MAE:  1.8588
CPU times: user 572 ms, sys: 12.8 ms, total: 585 ms
Wall time: 594 ms


In [8]:
%%time
class MeanAlgo(AlgoBase):

    def __init__(self):
        AlgoBase.__init__(self)

    def fit(self, trainset):
        AlgoBase.fit(self, trainset)
        self.the_mean = trainset.global_mean
        return self
        
    def estimate(self, u, i):
        return self.the_mean

algo = MeanAlgo()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

RMSE: 1.7194
MAE:  1.3530
CPU times: user 241 ms, sys: 7.97 ms, total: 249 ms
Wall time: 250 ms


In [5]:
%%time
algo = SVD()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

RMSE: 1.5259
MAE:  1.1656
CPU times: user 8.71 s, sys: 9.88 ms, total: 8.72 s
Wall time: 8.79 s


In [6]:
%%time
algo = SVDpp()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

RMSE: 1.5393
MAE:  1.1833
CPU times: user 17 s, sys: 20.6 ms, total: 17.1 s
Wall time: 17.2 s


In [21]:
%%time
algo = BaselineOnly()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions), accuracy.mae(predictions)

Estimating biases using als...
RMSE: 1.5357
MAE:  1.1724
CPU times: user 820 ms, sys: 10.9 ms, total: 831 ms
Wall time: 844 ms
